In [195]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [196]:
# Read .csv into a DataFrame and split by '\t'
elnino_lanina = pd.read_csv('../Data/Raw/El-Nino_La-Nina/El-Nino_La-Nina.csv', sep='\t')

# Set 'Season' as index (optional)
elnino_lanina.set_index('Season', inplace=True)

# Convert the entire DataFrame to integers (if needed)
elnino_lanina = elnino_lanina.astype(float)
elnino_lanina = elnino_lanina.reset_index()
elnino_lanina = elnino_lanina.rename(columns={'Season':'Years'})
elnino_lanina

,Years,JJA,JAS,ASO,SON,OND,NDJ,DJF,JFM,FMA,MAM,AMJ,MJJ
0,1950,-0.5,-0.4,-0.4,-0.4,-0.6,-0.8,-0.8,-0.5,-0.2,0.2,0.4,0.6
1,1951,0.7,0.9,1.0,1.2,1.0,0.8,0.5,0.4,0.3,0.3,0.2,0.0
2,1952,-0.1,0.0,0.2,0.1,0.0,0.1,0.4,0.6,0.6,0.7,0.8,0.8
3,1953,0.7,0.7,0.8,0.8,0.8,0.8,0.8,0.5,0.0,-0.4,-0.5,-0.5
4,1954,-0.6,-0.8,-0.9,-0.8,-0.7,-0.7,-0.7,-0.6,-0.7,-0.8,-0.8,-0.7
5,1955,-0.7,-0.7,-1.1,-1.4,-1.7,-1.5,-1.1,-0.8,-0.6,-0.5,-0.5,-0.5
6,1956,-0.6,-0.6,-0.5,-0.4,-0.4,-0.4,-0.2,0.1,0.4,0.7,0.9,1.1
7,1957,1.3,1.3,1.3,1.4,1.5,1.7,1.8,1.7,1.3,0.9,0.7,0.6
8,1958,0.6,0.4,0.4,0.4,0.5,0.6,0.6,0.6,0.5,0.3,0.2,-0.1
9,1959,-0.2,-0.3,-0.1,0.0,0.0,0.0,-0.1,-0.1,-0.1,0.0,0.0,0.0


In [197]:
# Melt to reshape the DataFrame with one row for each column
elnino_lanina = elnino_lanina.rename(columns={'JJA':'Jan','JAS':'Feb','ASO':'Mar','SON':'Apr','OND':'May',
                                              'NDJ':'June','DJF':'July','JFM':'Aug','FMA':'Sep','MAM':'Oct',
                                              'AMJ':'Nov','MJJ':'Dec'})

# Melt to reshape the DataFrame with one row for each column
elnino_lanina_melted = pd.melt(elnino_lanina, id_vars='Years', var_name='Month', value_name='Oceanic Nino Index')

# Groupby year and find the average of Oceanic Nino Index
oceanic_nino_index_avg = elnino_lanina_melted.groupby(['Years'])['Oceanic Nino Index'].mean().reset_index()

# Reset df name
elnino_lanina = oceanic_nino_index_avg

elnino_lanina

,Years,Oceanic Nino Index
0,1950,-0.283333
1,1951,0.608333
2,1952,0.350000
3,1953,0.375000
4,1954,-0.733333
5,1955,-0.925000
6,1956,0.008333
7,1957,1.291667
8,1958,0.416667
9,1959,-0.075000


In [198]:
elnino_lanina['El Nino or La Nina Year'] = ''

# From averages decide whether the year was an El Nino or La Nina 
elnino_lanina.loc[elnino_lanina['Oceanic Nino Index'] > 0, 'El Nino or La Nina Year'] = 'El Nino'
elnino_lanina.loc[elnino_lanina['Oceanic Nino Index'] <= 0, 'El Nino or La Nina Year'] = 'La Nina'
elnino_lanina

,Years,Oceanic Nino Index,El Nino or La Nina Year
0,1950,-0.283333,La Nina
1,1951,0.608333,El Nino
2,1952,0.350000,El Nino
3,1953,0.375000,El Nino
4,1954,-0.733333,La Nina
5,1955,-0.925000,La Nina
6,1956,0.008333,El Nino
7,1957,1.291667,El Nino
8,1958,0.416667,El Nino
9,1959,-0.075000,La Nina


In [199]:
# Create blank column to assign magnitude of index
elnino_lanina['Magnitude'] = ''

# Assign magnitude of index 
# https://ggweather.com/enso/oni.htm
def assign_magnitude(row):
    if row['El Nino or La Nina Year'] == 'El Nino':
        if 0.5 <= row['Oceanic Nino Index'] <= 0.9:
            return 'Weak'
        elif 1.0 <= row['Oceanic Nino Index'] <= 1.4:
            return 'Moderate'
        elif 1.5 <= row['Oceanic Nino Index'] <= 1.9:
            return 'Strong'
        elif row['Oceanic Nino Index'] >= 2:
            return 'Very Strong'
        else:
            return 'Neutral'
    elif row['El Nino or La Nina Year'] == 'La Nina':
        if -0.9 <= row['Oceanic Nino Index'] <= -0.5:
            return 'Weak'
        elif -1.4 <= row['Oceanic Nino Index'] <= -1.0:
            return 'Moderate'
        elif -1.9 <= row['Oceanic Nino Index'] <= -1.5:
            return 'Strong'
        elif row['Oceanic Nino Index'] <= -2:
            return 'Very Strong'
        else:
            return 'Neutral'
    else:
        return 'Unknown'

# Apply the function to create the 'Magnitude' column
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
elnino_lanina['Magnitude'] = elnino_lanina.apply(assign_magnitude, axis=1)


elnino_lanina



,Years,Oceanic Nino Index,El Nino or La Nina Year,Magnitude
0,1950,-0.283333,La Nina,Neutral
1,1951,0.608333,El Nino,Weak
2,1952,0.350000,El Nino,Neutral
3,1953,0.375000,El Nino,Neutral
4,1954,-0.733333,La Nina,Weak
5,1955,-0.925000,La Nina,Neutral
6,1956,0.008333,El Nino,Neutral
7,1957,1.291667,El Nino,Moderate
8,1958,0.416667,El Nino,Neutral
9,1959,-0.075000,La Nina,Neutral


In [200]:
# Save .csv to 'Cleaned' folder
elnino_lanina.to_csv('../Data/Cleaned/elnino_lanina.csv', index=False)